# Topology - Persistent Homology

In [39]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
from sklearn.metrics.pairwise import euclidean_distances
import gudhi as gd
import shutil
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

In [212]:
def get_rates(pair1, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, mt5.TIMEFRAME_M15, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1

AUDUSD = get_rates('AUDUSD.a', 175)

In [213]:
# Other data ideas:
# AUD basket, AUD cointegrating pairs, AUD correlated pairs
# arima values, garch values, arma values

In [214]:
df = AUDUSD[['open', 'high', 'low', 'close']]
df = df.iloc[:-1] 

In [215]:
df['tr1'] = df['high'] - df['low']
df['tr2'] = abs(df['high'] - df['close'].shift())
df['tr3'] = abs(df['low'] - df['close'].shift())
df['tr'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)

In [216]:
df['ma7'] = df['close'].rolling(window = 7).mean()
df['ma21'] = df['close'].rolling(window = 21).mean()
window_size = 7  # Choose the desired window size for the rolling calculation
df['atr'] = df['tr'].rolling(window=window_size).mean()
df = df.dropna()
df

,open,high,low,close,tr1,tr2,tr3,tr,ma7,ma21,atr
20,0.65478,0.65481,0.65398,0.65403,0.00083,0.00003,0.00080,0.00083,0.654903,0.654134,0.000766
21,0.65402,0.65433,0.65378,0.65388,0.00055,0.00030,0.00025,0.00055,0.654764,0.654134,0.000701
22,0.65389,0.65395,0.65201,0.65224,0.00194,0.00007,0.00187,0.00194,0.654349,0.654061,0.000871
23,0.65224,0.65248,0.65135,0.65153,0.00113,0.00024,0.00089,0.00113,0.653791,0.653935,0.000869
24,0.65154,0.65162,0.65108,0.65126,0.00054,0.00009,0.00045,0.00054,0.653249,0.653801,0.000851
...,...,...,...,...,...,...,...,...,...,...,...
169,0.64754,0.64770,0.64705,0.64723,0.00065,0.00015,0.00050,0.00065,0.647090,0.647552,0.000907
170,0.64723,0.64726,0.64597,0.64597,0.00129,0.00003,0.00126,0.00129,0.646971,0.647468,0.001004
171,0.64594,0.64632,0.64592,0.64619,0.00040,0.00035,0.00005,0.00040,0.646789,0.647390,0.000867
172,0.64619,0.64624,0.64583,0.64602,0.00041,0.00005,0.00036,0.00041,0.646629,0.647289,0.000760


In [217]:
# Compute the Euclidean distance matrix
dist_matrix = euclidean_distances(df)

# Use the GUDHI library to construct a Rips complex
rips_complex = gd.RipsComplex(distance_matrix=dist_matrix, max_edge_length=1.0)

simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# To visualize or analyze the topological features of the complex, you can use GUDHI's persistence diagram functionality:
diag = simplex_tree.persistence(min_persistence=0.01)

In [219]:
# Set a threshold for the minimum persistence that we consider significant
min_persistence = 0.1
threshold_high = 20
threshold_low = 5

# Calculate the persistence diagram
diag = simplex_tree.persistence(min_persistence=min_persistence)

# Extract the persistent features (those with persistence above the threshold)
persistent_features = [interval for interval in diag if interval[1][1] - interval[1][0] > min_persistence]

# Hypothetical trading logic
if len(persistent_features) > threshold_high:
    print("The market is complex, potentially indicating a trend change. Short")
    if len(mt5.positions_get()) == 0:
        sell_order()
    else:
        for i in mt5.positions_get():
            if 'Rips Complex' in i.comment and i.type == 0:
                position = i 
                close_position(position) ## still closing everything else, amend when important
            

elif len(persistent_features) < threshold_low:
    print("The market is simple, potentially indicating a trend continuation. Buy")
    if len(mt5.positions_get()) == 0:
        buy_order()
    else:
        for i in mt5.positions_get():
            close_position(i)
else:
    print("The market is in an intermediate state. No action taken.")

The market is simple, potentially indicating a trend continuation. Buy


In [71]:
def buy_order():
    price = mt5.symbol_info_tick('AUDUSD.a').ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": 'AUDUSD.a',
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": f"Rips Complex",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1
    
def sell_order():
    price = mt5.symbol_info_tick('AUDUSD.a').bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": 'AUDUSD.a',
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": f"Rips Complex",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1
    
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'pytohn script close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result1 = mt5.order_send(request)